# CU Recommender System

In this example we are creating a cu recommender system by extractung feature vectors using PaddlePaddle, importing bets vectors into Milvus, and then searching in Milvus and Redis.

## Data
In this project, we use qa3 tenant data for 2021. This dataset contains approximately 4900 cus made under 160 Books and 2500 solutions. 

We use the following files:
- cu.dat: Contains Gsi -> Cus mapping information.
- bet.dat: Contains Cu -> GEs mapping information.


## Requirements

Due to package constraints, this notebook needs to be run using Python 3.6/3.7 . It is recommended that you use a virtual enviroment like Conda, instructions for installing Conda can be found [here](https://conda.io/projects/conda/en/latest/user-guide/install/index.html). 

Currently, there is a dirty workaround that you can use for python 3.8+. When installing the `requirements.txt`, pip will fail to install`sentencepiece`. If you rerun the notebook after the install fails and avoid redownloading the packages, the rest of the notebook should run without any hiccups.

|  Packages |  Servers |
| --------------- | -------------- |
| pymilvus==2.0.0rc5 | milvus-2.0.0-rc5 |
| pymongo           | mongodb          |
| paddle_serving_app |
| paddlepaddle==2.1.1 |


We have included a requirements.txt file in order to easily satisfy the required packages. 

## Up and Running

### Installing Packages
Install the required python packages with `requirements.txt`. If using Python 3.8, look at workaround in the Requirements section. Uninstall previous pymilvus-orm if needed.

In [ ]:
! pip3 install -r requirements.txt

### Getting Milvus Server

This demo uses Milvus 2.0 Standalone with docker-compose, please refer to [Install Milvus 2.0](https://milvus.io/docs/v2.0.0/install_standalone-docker.md) for other installation options (on Kubernetes or use Milvus Cluster). 

Alternatively we have deployed standalone milvus in dev kubernetes. We need to grep the endpoint by pod ip.
This can be done by
`kubectl get po -o wide | grep milvus-standalone`

In [ ]:
!helm install milvus milvus/milvus --set cluster.enabled=false \
--set minio.persistence.storageClass=ebs-sc --set log.persistence.persistentVolumeClaim.storageClass=ebs-sc \
--set standalone.persistence.persistentVolumeClaim.storageClass=ebs-sc --set minio.persistence.size=50Gi \
--set etcd.global.storageClass=ebs-sc --set pulsar.default_storage.existingStorageClassName=ebs-sc

In [ ]:
#!docker-compose up -d

### Starting Redis Server
We are using Redis as a metadata storage service. Code can easily be modified to use a python dictionary, but that usually does not work in any use case outside of quick examples. We need a metadata storage service in order to be able to be able to map between embeddings and the corresponding data.

In [ ]:
!docker run  --name redis -d -p 6379:6379 redis

### Confirm Running Servers

In [ ]:
! docker ps

### Downloading Data from Mongodb (Bets Store)

In [ ]:
import pandas as pd
import urllib.parse
from pymongo import MongoClient

def _connect_mongo(host, port, username, password, db):
    """ A util for making a connection to mongo """

    if username and password:
        mongo_uri = 'mongodb://%s:%s@%s:%s/%s?authSource=%s&readPreference=primary&ssl=false' % (username, urllib.parse.quote(password), host, port, db,'nsl_bet_db_qa3')
        conn = MongoClient(mongo_uri)
    else:
        conn = MongoClient(host, port)

    return conn[db]


In [ ]:
def read_mongo(db, collection, query={}, host='10.220.98.254', port=27017, username='bet', password='bet@123', no_id=False):
    """ Read from Mongo and Store into DataFrame """

    # Connect to MongoDB
    db = _connect_mongo(host=host, port=port, username=username, password=password, db=db)

    # Make a query to the specific DB and Collection
    cursor = db[collection].aggregate(query)

    # Expand the cursor and construct the DataFrame
    df =  pd.DataFrame(list(cursor))
    
    # Delete the _id
    if no_id:
        del df['_id']

    return df

In [ ]:
gsi_pipeline = [
    {
        "$project": {"nb": "$$ROOT", "_id": 0}
    },
    {
        "$lookup": {
            "localField": "nb.gsiList.id",
            "from": "nsl_gsi",
            "foreignField": "id",
            "as": "ng"
        }
    },
    {
        "$unwind": {
            "path": "$ng",
            "preserveNullAndEmptyArrays": False
        }
    },
    {
        "$lookup": {
            "localField": "ng.solutionLogic.referencedChangeUnit",
            "from": "nsl_change_unit",
            "foreignField": "id",
            "as": "cu"
        }
    },
    {
        "$unwind": {
            "path": "$cu",
            "preserveNullAndEmptyArrays": False
        }
    },
    {
        "$addFields": {
            "bookId": "$nb.displayName",
            "gsiId": "$ng.displayName",
            "bookName": "$nb.displayName",
            "gsiName": "$ng.displayName",
            "cuName": "$cu.displayName"
        }
    },
    {
        "$match": {
            "$and": [
                {
                    "nb.tenantId": {
                        "$in": [
                            "ProjectCarnivals",
                            "Banking",
                            "projectmanagement",
                            "Brane-Finance",
                            "FinanceSolBrane",
                            "AILABS",
                            "BRF2008",
                            "Finance2008",
                            "FamilyApp2008",
                            "Learning2008",
                            "GRC",
                            "Insurance",
                            "Healthcare",
                            "SupplyChain",
                            "Pharma",
                            "CustomerSuccess"]
                    }
                },
                {"nb.displayName": {"$regex": "^((?!test).)*$", "$options": "i"}},
                {"nb.displayName": {"$regex": "^((?!book).)*$", "$options": "i"}},
                {"ng.displayName": {"$regex": "^((?!test).)*$", "$options": "i"}}
            ]
        }
    },
    {
        "$group": {
            "_id": {"bookId": "$bookId", "gsiId": "$gsiId"},
            "addToSet(cu_name)": {"$addToSet": "$cu.displayName"}
        }
    },
    {
        "$project": {
            "bookId": "$_id.bookId",
            "gsiId": "$_id.gsiId",
            "cuId": "$addToSet(cu_name)",
            "bookName": "$nb.displayName",
            "gsiName": "$ng.displayName",
            "cuName": "$cu.displayName",
            "_id": 0
        }
    }
]

In [ ]:
df = read_mongo(db='nsl_bet_db_soln',collection='nsl_book',query=gsi_pipeline)
df.to_pickle("data/cu.pickle")

## Code Overview


### Importing Gsi into Milvus

#### 1. Connectings to Milvus and Redis
Both servers are running as Docker containers on the localhost with their corresponding default ports.

In [ ]:
from pymilvus import *
import redis

# connections.connect()
connections.connect("default", host="10.100.22.245", port="19530")

EXPIRATION_SECONDS = 600
r = redis.StrictRedis(host="localhost", port=6379) 


#### 2. Loading CU into Redis
We begin by loading all the movie files into redis. 

In [ ]:
import pandas as pd

df2 = pd.read_pickle("data/cu.pickle")
#keys = df2[["gsiId","bookId","cuId"]]
#print(keys.to_dict('records'))
#r.hmset("gsiId",keys.to_dict('records'))

#### 3.1 Creating Partition and Collection in Milvus

In [ ]:
COLLECTION_NAME = 'demo_cu'
PARTITION_NAME = 'GsiCu'

pk = FieldSchema(name='id', dtype=DataType.INT64, is_primary=True, auto_id=True)
field = FieldSchema(name='gsiId', dtype=DataType.FLOAT_VECTOR, dim=768)
schema = CollectionSchema(fields=[pk, field], description="cu recommender: demo_cu")

if utility.get_connection().has_collection(COLLECTION_NAME): # drop the same collection created before
    collection = Collection(COLLECTION_NAME)
    collection.drop()
else:
    collection = Collection(name=COLLECTION_NAME, schema=schema)
    partition = collection.create_partition(PARTITION_NAME)
    print("Collection & partition are successfully created.")

#### 3.2 Setting an Index
After creating the collection we want to assign it an index type. This can be done before or after inserting the data. When done before, indexes will be made as data comes in and fills the data segments. In this example we are using IVF_SQ8 which requires the 'nlist' parameter.

In [ ]:
# Flush collection with inserted vectors to disk
# utility.get_connection().flush([COLLECTION_NAME])

index_param = {
        "metric_type":"L2",
        "index_type":"IVF_SQ8",
        "params":{"nlist":1024}
    }
collection.create_index(field_name='gsiId', index_params=index_param)

#### 4.1 Generating Embeddings
In this example we are using the sentence_transformer library to encode the sentence into vectors. This library uses a modified BERT model to generate the embeddings, and in this example we are using a model pretrained using Microsoft's mpnet. More info can be found here.

In [ ]:
from sentence_transformers import SentenceTransformer
import pandas as pd
from sklearn.preprocessing import normalize

model = SentenceTransformer('paraphrase-mpnet-base-v2')
# Get questions and answers.
title_data = df2['gsiId'].tolist()
text_data = df2['cuId']

sentence_embeddings = model.encode(title_data)
sentence_embeddings = normalize(sentence_embeddings)
print(type(sentence_embeddings))

#### 4.2 Getting Embeddings and IDs
Since current dataset contains only 550 vectors, we are inserting all of them as one batch insert.

In [ ]:
embeddings =list(sentence_embeddings.head(10))

#### 4. Importing Vectors into Milvus
Import vectors into the partition **GsiCu** under the collection **demo_cu**.

In [50]:
if collection.num_entities != 0:
    print(COLLECTION_NAME + " is not empty!")  
else:
    mr = collection.insert(data=[embeddings], partition_name=PARTITION_NAME)

print("Record count in collection: " + str(collection.num_entities))
print(str(len(mr.primary_keys)) + " ids:\n", mr.primary_keys[:10])

### Recalling Vectors in Milvus
#### 1. Genarating User Embeddings
Pass in the gender, age and occupation of the user we want to recommend. **user_vector_model** model will generate the corresponding user vector.
Occupation is chosen from the following choices:
*  0:  "other" or not specified
*  1:  "academic/educator"
*  2:  "artist"
*  3:  "clerical/admin"
*  4:  "college/grad student"
*  5:  "customer service"
*  6:  "doctor/health care"
*  7:  "executive/managerial"
*  8:  "farmer"
*  9:  "homemaker"
*  10:  "K-12 student"
*  11:  "lawyer"
*  12:  "programmer"
*  13:  "retired"
*  14:  "sales/marketing"
*  15:  "scientist"
*  16:  "self-employed"
*  17:  "technician/engineer"
*  18:  "tradesman/craftsman"
*  19:  "unemployed"
*  20:  "writer"

In [ ]:
import numpy as np
from paddle_serving_app.local_predict import LocalPredictor

class RecallServerServicer(object):
    def __init__(self):
        self.uv_client = LocalPredictor()
        self.uv_client.load_model_config("movie_recommender/user_vector_model/serving_server_dir") 
        
    def hash2(self, a):
        return hash(a) % 1000000

    def get_user_vector(self):
        dic = {"userid": [], "gender": [], "age": [], "occupation": []}
        lod = [0]
        dic["userid"].append(self.hash2('0'))
        dic["gender"].append(self.hash2('M'))
        dic["age"].append(self.hash2('23'))
        dic["occupation"].append(self.hash2('6'))
        lod.append(1)

        dic["userid.lod"] = lod
        dic["gender.lod"] = lod
        dic["age.lod"] = lod
        dic["occupation.lod"] = lod
        for key in dic:
            dic[key] = np.array(dic[key]).astype(np.int64).reshape(len(dic[key]),1)
        fetch_map = self.uv_client.predict(feed=dic, fetch=["save_infer_model/scale_0.tmp_1"], batch=True)
        return fetch_map["save_infer_model/scale_0.tmp_1"].tolist()[0]

recall = RecallServerServicer()
user_vector = recall.get_user_vector()

#### 2. Searching
Pass in the user vector, and then recall vectors in the previously imported data collection and partition.

In [ ]:
collection.load() # load collection memory before search

topK = 20
SEARCH_PARAM = {
    "metric_type":"L2",
    "params":{"nprobe": 20},
    }
results = collection.search([user_vector],"vec",param=SEARCH_PARAM, limit=topK, expr=None, output_fields=None)

#### 3. Returning Information by IDs

In [ ]:
I = []
for x in results:
    for y in x.ids:
        I.append(y)
        
recall_results = []
for x in I:
    recall_results.append(r.get("{}##movie_info".format(x)).decode('utf-8'))
recall_results

## Conclusion

After completing the recall service, the results can be further sorted using the **movie_recommender** model, and then the movies with high similarity scores can be recommended to users. You can try this deployable recommender system using this [quick start](QUICK_START.md).